# Install

In [ ]:
!pip install datasets -q
!pip install transformers -q
!pip install seqeval -q
#!pip install wandb -q
!pip install emoji -q

     |████████████████████████████████| 451 kB 4.6 MB/s 
     |████████████████████████████████| 132 kB 82.2 MB/s 
     |████████████████████████████████| 182 kB 77.6 MB/s 
     |████████████████████████████████| 212 kB 91.8 MB/s 
     |████████████████████████████████| 127 kB 82.3 MB/s 
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 65.6 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 240 kB 4.4 MB/s 


In [ ]:
!python

In [ ]:
!ls drive/MyDrive/CorpusProfNer/

In [ ]:
datapath='drive/MyDrive/CorpusProfner/'
valid_file= datapath+'valid_clean.txt'

training_file = datapath+'train_50_mr.txt'
folder_name= "test6"


In [ ]:
!nvidia-smi

Thu Dec 15 20:15:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    52W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Dataset

In [ ]:
import pandas as pd
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch



def read_bio_dataset(dir):
  tok = []  #Aux list of tokens for current sentence
  bio = []  #Aux list of ner tags for current sentence
  df_list = []  #Final list with all the information

  with open(dir,'r',encoding='utf-8') as file:
    for line in file.readlines():

      #When reaching the end of a sentence, we append and restart tok and bio
      #We also check for non-empty sentences
      if line == '\n' and tok!=[] and bio!=[]:
        df_list.append([tok,bio])
        tok = []
        bio = []

      else:

        #We add the token and ner_tag to the list
        tok.append(line.split(' ')[0])
        bio.append(line.split(' ')[-1].replace('\n',''))

  #Returning df_list to a dataframe
  return pd.DataFrame(df_list, columns=['tokens','ner_tags'])

import emoji




def read_tsv_dataset(name):
  training_data= pd.read_csv(name, sep="\t",encoding='utf8')
  training_data['tokens'] = training_data['tokens'].apply(eval)
  training_data['ner_tags'] = training_data['ner_tags'].apply(eval)
  return training_data





'''
write_bio_dataset(valid_data,'valid_formated.txt')
write_bio_dataset(training_data,'train_formated.txt')


valid_data = read_bio_dataset('valid_spacy.txt')
valid_data = clean_data(valid_data)
valid_data.head(26)

training_data = read_bio_dataset('train_spacy.txt')
training_data = clean_data(training_data)
training_data.head(26)


valid_data = read_bio_dataset('valid_formated.txt')
training_data = read_bio_dataset('train_formated.txt')

valid_data = clean_data(valid_data)
training_data = clean_data(training_data)

'''


valid_data = read_bio_dataset(valid_file)

training_data= read_bio_dataset(training_file)





'''


training_data= pd.read_csv('training_or.tsv', sep="\t", encoding='utf8')
training_data['tokens'] = training_data['tokens'].apply(eval)
training_data['ner_tags'] = training_data['ner_tags'].apply(eval)

training_data = clean_data(training_data)
training_data



'''
train_dataset = Dataset.from_pandas(training_data)
test_dataset = Dataset.from_pandas(valid_data)








labels_list = ['O', 'B-PROFESION', 'I-PROFESION']
label_num_list= list(range(0,len(labels_list)))




label2id={}
id2label={}
for label,num in zip(labels_list,label_num_list):
  label2id[label]=num
  id2label[num]=label






task = "ner"

model_checkpoint = "PlanTL-GOB-ES/roberta-base-bne"


tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True, truncation=True,  max_length=512)



def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]),  truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == 'O':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])#(label2id[label[word_idx]])
                #label_ids.append(label[word_idx])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)#(label2id[label[word_idx]] if label_all_tokens else -100)
                #label_ids.append(label[word_idx] if label_all_tokens else -100)#(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs



train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
#valid_tokenized_datasets = valid_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)



from transformers import  RobertaForTokenClassification, AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(labels_list), id2label = id2label, label2id = label2id)


batch_size = 16
epochs= 6
args = TrainingArguments(
    folder_name,
    evaluation_strategy = "epoch",
    save_strategy="no",

    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,

    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    learning_rate=1e-4,
    #fp16=True,
    #"weight_decay": (0, 0.3),
  #"learning_rate": (1e-5, 5e-5),



    optim="adamw_torch"

   # report_to="wandb" ## WANDB

)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    #print(predictions)

    true_predictions = [[labels_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[labels_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}



trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()




loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--PlanTL-GOB-ES--roberta-base-bne/snapshots/6f11dfab050340aebc487ccf58ce349b8f88bc67/config.json
Model config RobertaConfig {
  "_name_or_path": "PlanTL-GOB-ES/roberta-base-bne",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_length": 512,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50262
}

loading file vocab.json from cache at /root/.cache/huggingface/hub/mo

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--PlanTL-GOB-ES--roberta-base-bne/snapshots/6f11dfab050340aebc487ccf58ce349b8f88bc67/config.json
Model config RobertaConfig {
  "_name_or_path": "PlanTL-GOB-ES/roberta-base-bne",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PROFESION",
    "2": "I-PROFESION"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-PROFESION": 1,
    "I-PROFESION": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.102200,0.025206,0.727869,0.644412,0.683603,0.993324
2,0.012800,0.036358,0.691740,0.680697,0.686174,0.993109
3,0.003100,0.033699,0.709585,0.719884,0.714697,0.993756
4,0.001600,0.039101,0.756800,0.686502,0.719939,0.993921
5,0.000900,0.033996,0.752624,0.728592,0.740413,0.994327
6,0.000200,0.039626,0.716954,0.724238,0.720578,0.993629


The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3693
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3693
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: ner_tags, tokens. If ner_tags, tokens are not expected by `RobertaForTokenClassification.forward`,  you can safely ignore this message.
***** Runnin

TrainOutput(global_step=4146, training_loss=0.01540013221782141, metrics={'train_runtime': 364.5775, 'train_samples_per_second': 181.854, 'train_steps_per_second': 11.372, 'total_flos': 1942150613526576.0, 'train_loss': 0.01540013221782141, 'epoch': 6.0})

In [ ]:
import transformers
transformers.__version__

'4.25.1'

In [ ]:
import datasets
datasets.__version__

'2.7.1'

In [ ]:
!pip freeze

In [ ]:
def get_data_val()

In [ ]:
train_tokenized_datasets

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 11052
})

In [ ]:
train_tokenized_datasets[0]

In [ ]:
write_tokenizeddataset(train_tokenized_datasets,'train_full.txt')

In [ ]:
def write_tokenizeddataset(dataset,outputfile):
  with open(outputfile, 'w') as f:

    for index in range(0,len(dataset)):
      toks= dataset[index]['input_ids']
      tags= dataset[index]['labels']
      f.write(str(toks)+'\n')
      f.write(str(tags)+'\n')
      f.write('\n')




#Saving

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()


In [ ]:
!rm -r profner_model

In [ ]:
trainer.evaluate()
trainer.save_model('profner1')

In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()

#Excuting

In [ ]:
!pip install pyocclient -q


In [ ]:
import owncloud
oc = owncloud.Client('https://delicias.dia.fi.upm.es/nextcloud/')
#oc.login('asanchez', 'AS.sczz.448')
oc.login('pcalleja', 'oWn.ser.5')

In [ ]:
!zip -r ./base.model-5.zip ./base.model


In [ ]:
oc.put_file('base-model-5.zip', 'base.model-5.zip')


In [ ]:
oc.get_file('ProfNer/training_or.tsv', 'training_or.tsv')


In [ ]:
oc.get_file('ProfNer/training_50.tsv', 'training_50.tsv')
oc.get_file('ProfNer/training_30.tsv', 'training_30.tsv')
oc.get_file('ProfNer/training_10.tsv', 'training_10.tsv')
oc.get_file('ProfNer/train_spacy.txt', 'train_spacy.txt')
oc.get_file('ProfNer/valid_spacy.txt', 'valid_spacy.txt')

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("profner1")
model = AutoModelForTokenClassification.from_pretrained("profner1")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)


In [ ]:
example = "hola conductores de ambulancia y viva la guardia civil"

ner_results = nlp(example)
print(ner_results)

[]


In [ ]:
nlp('Regarding Mossack Fonseca S.A.')